In [18]:
import numpy as np
import pandas as pd
import pymssql

import xlwt

import pandas.io.formats.excel

#import smtplib

In [38]:
# 定义净现值函数
def npv(CF,discount_rate):
    #折现时间段的折现率
    #s 是 Cash Flow Statement
    # discount_rate 折现率
    import math
    import numpy as np
    rate=np.ones(shape=(13))
    n=0
    for r in rate:
        r=1/math.pow(1+discount_rate,n)
        rate[n]=r
        n=n+1
    NPV=CF*rate
    return NPV.sum()

def main():
    # key represents 项目敏感性相关的变量 包含：成本，投资，产量，价格
    # df represents  项目敏感性的表格
    # change_rate represents 折扣 

    # 设置变量
    change_rate = np.array([0.95,0.97,1,1.03,1.05])
    var_len=change_rate.size
    change_in_IRRNPV_vol = pd.Series(np.zeros(shape=var_len),change_rate) #IRR change with Volume
    change_in_IRRNPV_cost = pd.Series(np.zeros(shape=var_len),change_rate) #IRR change with Cost
    change_in_IRRNPV_price = pd.Series(np.zeros(shape=var_len),change_rate) #IRR change with Price
    change_in_IRRNPV_investment = pd.Series(np.zeros(shape=var_len),change_rate) #IRR change with Investment

    dic_senstivity={'Volume':change_in_IRRNPV_vol,'Cost':change_in_IRRNPV_cost,'Price':change_in_IRRNPV_price,'Investment':change_in_IRRNPV_investment}
    IRR_senstivity_sheet = pd.DataFrame(dic_senstivity)
    NPV_senstivity_sheet = pd.DataFrame(dic_senstivity)
    IRR_senstivity_sheet[['Volume','Cost','Price','Investment']]=IRR_senstivity_sheet[['Volume','Cost','Price','Investment']].apply(lambda x: round(x,2))
    NPV_senstivity_sheet[['Volume','Cost','Price','Investment']]=NPV_senstivity_sheet[['Volume','Cost','Price','Investment']].apply(lambda x: round(x,2))
    
    # 准备Cash Flow的表格
    pandas.io.formats.excel.header_style = None
    writer3 = pd.ExcelWriter(r'C:\Users\huxinrui\Documents\Python Scripts\Cash_Flow_sheet.xlsx')
    for k in dic_senstivity.keys():
        dic_coe = {'Volume':1,'Cost':1,'Price':1,'Investment':1}
        vp_cost_perUnit=0.5 #可变制造成本理论上随产量变化而变化，但是咱们的例子可以假设不变，很难降本
        r_cost =3000 #房租成本，一般不随产量变化，也很难降本

        num = 0 #开始计数    

        for coe in change_rate:

            dic_coe[k] = coe 
            vol = 3600 * dic_coe['Volume']           #产量 产量可变
            m_cost_perUnit = 2.195 * dic_coe['Cost'] #材料成本 可以降本，购买低廉的材料   c_cost
            equipment_cost=30000 * dic_coe['Investment']#三年straight-line depreciation
            l_cost = 4000 * dic_coe['Cost']          #卖烧饼的人员工资，一般不随产量变化，可以降本  c_cost
            #e_cost = equipment_cost / 36# 购买设备成本，可以降本，买便宜的设备 * c_invest
            price = 6.85 * dic_coe['Price']              #可变售价 * c_price



            report_date = pd.Series(pd.date_range('20201215','20220115',freq='1M')) #财务时间周期

            CF_sheet = create_sheet(report_date,vol,price,m_cost_perUnit,vp_cost_perUnit,l_cost,r_cost,equipment_cost)
            set_excel(CF_sheet,writer3,'CF'+k+str(num))   

            # 计算NPV净现值  
            NPV_senstivity_sheet[k].iloc[num]=npv(CF_sheet['CF Net Income'],0.00948)    
            # 计算IRR 内部收益率
            IRR_senstivity_sheet[k].iloc[num] =np.irr(CF_sheet['CF Net Income'])
            num=num+1
        IRR_senstivity_sheet[k]=IRR_senstivity_sheet[k].apply(lambda x: round(x,2))
        NPV_senstivity_sheet[k]=NPV_senstivity_sheet[k].apply(lambda x: round(x,2))   
    writer3.save()
    writer3.close() 
    print(IRR_senstivity_sheet,'\n',NPV_senstivity_sheet)
# 原因是Python中 \ 是转义符，\u表示后面是UNICODE编码，所以\User在这里会报错。所以在字符串前面加个 r就可以解决了。


    pandas.io.formats.excel.header_style = None
    writer1 = pd.ExcelWriter(r'C:\Users\huxinrui\Documents\Python Scripts\IRR_senstivity_sheet.xlsx')
    set_excel(IRR_senstivity_sheet,writer1,'IRR')
     # 输出表格
    writer1.save()
    writer1.close() 
    pandas.io.formats.excel.header_style = None
    writer2 = pd.ExcelWriter(r'C:\Users\huxinrui\Documents\Python Scripts\NPV_senstivity_sheet.xlsx')
    set_excel(NPV_senstivity_sheet,writer2,'NPV')    
     # 输出表格
    writer2.save()
    writer2.close() 
    

    



In [39]:
 def create_sheet(report_date,vol,price,m_cost_perUnit,vp_cost_perUnit,l_cost,r_cost,equipment_cost):
        #------------------------------------------------------------------------------------
        #名词解释
        # report_date     财务时间周期
        # vp_cost_perUnit 可变制造成本理论上随产量变化而变化
        # r_cost          房租成本
        # vol            产量 
        # m_cost_perUnit 材料成本 
        # equipment_cost 设备三年straight-line depreciation
        # l_cost         卖烧饼的人员工资
        # price          可变售价 
        #------------------------------------------------------------------------------------
                
        e_cost = equipment_cost / 36# 购买设备成本，可以降本，买便宜的设备 * c_invest
        duration = report_date.size
        revenue=pd.Series(np.zeros(shape=(duration)),report_date)#每月收益
        m_cost=pd.Series(np.zeros(shape=(duration)),report_date) #每月材料成本比
        vp_cost=pd.Series(np.zeros(shape=(duration)),report_date) #每月可变成本
        gross_profit=pd.Series(np.zeros(shape=(duration)),report_date) #每月毛利润
        fixed_cost =pd.Series(np.zeros(shape=(duration)),report_date) #每月固定成本
        EBIT=pd.Series(np.zeros(shape=(duration)),report_date)#每月含税息净利润 

        #构建利润表的每一个数据条
        revenue[1:]=revenue[1:]+vol*price
        m_cost[1:]=m_cost[1:]+m_cost_perUnit*vol
        gross_profit=revenue-m_cost
        vp_cost[1:]=vp_cost[1:]+vp_cost_perUnit*vol
        fixed_cost[1:]=fixed_cost[1:]+l_cost+r_cost+e_cost
        fixed_cost[0] = r_cost
        EBIT=gross_profit-vp_cost-fixed_cost


        dic_IS={'Revenue':revenue, 'MaterialCost':m_cost,'Gross Profit':gross_profit,'Variable Cost':vp_cost,'Fixed Cost':fixed_cost
            ,'EBIT':EBIT}

        income_statement = pd.DataFrame(dic_IS)
        #修改打印参数
        income_statement[['Variable Cost','Fixed Cost','EBIT']]=income_statement[['Variable Cost','Fixed Cost','EBIT']].apply(lambda x: round(x,2))


        # 构建现金流量表
        investment_CF=pd.Series(np.zeros(shape=duration),report_date)
        investment_CF[0]=-equipment_cost
        rent_CF=pd.Series(np.zeros(shape=13),report_date)
        rent_CF[0]=-r_cost*4
        rent_CF[3]=-r_cost*3
        rent_CF[6]=-r_cost*3
        rent_CF[9]=-r_cost*3
        depreciation = pd.Series(np.zeros(shape=duration),report_date)+e_cost
        depreciation[0]=0
        rent =  pd.Series(np.zeros(shape=duration),report_date)+r_cost
        net_income = pd.Series(np.zeros(shape=duration),report_date)+EBIT+investment_CF+rent_CF+depreciation+rent

        dic_CFS={'EBIT':EBIT,
                 'Onetime Investment':investment_CF,
                 'Rent CF Out':rent_CF,
                 'Backtrack Rent':rent,
                 'Depreciation':depreciation,
                'CF Net Income':net_income}


        CF_statement=pd.DataFrame(dic_CFS)
        CF_statement[['EBIT','Depreciation']]=CF_statement[['EBIT','Depreciation']].apply(lambda x:round(x,2))
        return CF_statement

In [42]:
def set_excel(df,writer,name):

   

    df.to_excel(writer, name)
    
    # -----------------------------------以下代码段增加自定义格式 ------------------------------------------

    # 设置格式

    workbook1 = writer.book

    worksheets = writer.sheets

    worksheet1 = worksheets[name]

    # 设置特定单元格的宽度

    worksheet1.set_column("N:N", 20)

    worksheet1.set_column("A:A", 30)

    worksheet1.set_column("H:H", 9)

    worksheet1.set_column("L:L", 9)
    
    

    # 我们在操作worksheet中的各自前，需要往workbook当中添加自定义格式集合：
    # format1 = workbook1.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
    # format2 = workbook1.add_format({'bold':  True, 'align': 'left', 'valign': 'top', 'text_wrap': True})
    # 新增格式：
    # bold = workbook1.add_format({
    #   'bold':  True,  # 字体加粗
    #   'border': 1,  # 单元格边框宽度
    #   'align': 'left',  # 水平对齐方式
    #    'valign': 'vcenter',  # 垂直对齐方式
    #    'fg_color': '#F4B084',  # 单元格背景颜色
    #    'text_wrap': True,  # 是否自动换行
    # })
    # 新增单元格数据：
    # //row:行， col：列， data:要写入的数据, bold:单元格的样式
    # worksheet1.write(row, col, data, bold)
    # 更多设置请参考 https://www.lmlphp.com/user/2614/article/item/33465/
    # 官方Docu： https://xlsxwriter.readthedocs.io/worksheet.html
    
    
    
    # 修改几个特殊单元格的内容

    cell_format = workbook1.add_format({'bold': True})

    worksheet1.write('A1', 'Variant', cell_format)


    # 标题加粗

    format1 = workbook1.add_format({'bold': 1})

    format2 = workbook1.add_format({'left':6})

    worksheet1.conditional_format('B1:N2', {'type':     'cell',

                                    'criteria': '>',

                                    'value':    15,

                                    'format':   format1

                                           })

    # 隐藏线条和第三行

    worksheet1.hide_gridlines(option=2)

    #worksheet1.set_row(2, None, None, {'hidden': True})
    
    # -----------------------------------以上代码段增加自定义格式 ------------------------------------------

   

In [43]:
main()

C:\Users\huxinrui\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: numpy.irr is deprecated and will be removed from NumPy 1.20. Use numpy_financial.irr instead (https://pypi.org/project/numpy-financial/).


      Volume  Cost  Price  Investment
0.95    0.16  0.20   0.14        0.19
0.97    0.16  0.19   0.16        0.18
1.00    0.18  0.18   0.18        0.18
1.03    0.19  0.17   0.20        0.17
1.05    0.20  0.16   0.21        0.17 
         Volume      Cost     Price  Investment
0.95  47773.17  62938.52  42295.35    57718.57
0.97  51151.33  60250.54  47864.64    57118.57
1.00  56218.57  56218.57  56218.57    56218.57
1.03  61285.80  52186.59  64572.49    55318.57
1.05  64663.96  49498.61  70141.78    54718.57


In [31]:
a=2


In [32]:
str(a)

'2'

In [33]:
'1'+str(a)

'12'